In [ ]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc);    

In [ ]:
val carRegistration = sqlContext.read.format("csv").option("header", "true").load("./car_registration.csv")

In [ ]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc);    

val carRegistration = sqlContext.read.format("csv").option("header", "true").load("./car_registration.csv")

carRegistration.take(1)

case class info( speed: String, travel_time: String, date: String, link_id: String, link_points: String, encoded_poly_line: String, encoded_polyline_lvls: String, borough: String, link_name: String)

val dateFormat1 = new java.text.SimpleDateFormat("yyyy MMM dd hh:mm:ss")
val dateFormat2 = new java.text.SimpleDateFormat("MM/dd/yyyy hh:mm:ss a")
val outputformat = new java.text.SimpleDateFormat("MM-dd-yyyy HH");
val contentRdd = carRegistration.filter( row => row.getAs[String](3).toInt >= 0).rdd
                .map((row) => info(
                                    row.getAs[String](1),//speed
                                    row.getAs[String](2), //travel_time
                                    if(row.getAs[String](4).size >= 23) 
                                    outputformat.format(dateFormat1.parse(row.getAs[String](4))) 
                                    else outputformat.format(dateFormat2.parse(row.getAs[String](4))), //date
                                    row.getAs[String](5), //link_id
                                    row.getAs[String](6), //link_points
                                    row.getAs[String](7), //encoded_poly_line
                                    row.getAs[String](8), //encoded_poly_line_lvls
                                    row.getAs[String](11).toLowerCase, //borough
                                    row.getAs[String](12) //link_name
                                                       ))

val pattern = """(?<=\()[^)]+(?=\))""".r

case class neighborhood(latitude: Double, longitude: Double, name: String, borough: String)

def deg2rad(deg: Double):Double ={
  deg * (Math.PI/180)
}
def getDistanceFromLatLonInKm(lat1: Double, lon1: Double, lat2: Double,lon2: Double):Double ={
  val R = 6371; // Radius of the earth in km
  val dLat = deg2rad(lat2-lat1);  // deg2rad below
  val dLon = deg2rad(lon2-lon1); 
  val a = 
    Math.sin(dLat/2) * Math.sin(dLat/2) +
    Math.cos(deg2rad(lat1)) * Math.cos(deg2rad(lat2)) * 
    Math.sin(dLon/2) * Math.sin(dLon/2)
    ; 
  val c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a)); 
  val d = R * c; // Distance in km
  R * c
}

def convertNeighborhood(row: org.apache.spark.sql.Row): neighborhood ={
  
  neighborhood(row.getAs[String](1).toDouble, row.getAs[String](2).toDouble, row.getAs[String](0).toLowerCase, row.getAs[String](3).toLowerCase)
}

val barrios = sqlContext.read.format("csv").option("header", "true").load("./Neighborhoods.csv")
barrios.createOrReplaceTempView("export_csv")


val bronx = spark.sql("select Name, latitude, longitude, Borough from export_csv where Borough == \"Bronx\"").rdd.map( convertNeighborhood(_) ).collect().toList
val manhattan = spark.sql("select Name, latitude, longitude, Borough from export_csv where Borough == \"Manhattan\"").rdd.map( convertNeighborhood(_) ).collect().toList
val queens = spark.sql("select Name, latitude, longitude, Borough from export_csv where Borough == \"Queens\"").rdd.map( convertNeighborhood(_) ).collect().toList
val brooklyn = spark.sql("select Name, latitude, longitude, Borough from export_csv where Borough == \"Brooklyn\"").rdd.map( convertNeighborhood(_) ).collect().toList
val statenIsland = spark.sql("select Name, latitude, longitude, Borough from export_csv where Borough == \"Staten Island\"").rdd.map( convertNeighborhood(_) ).collect().toList

def menor(x:(Double, String) ,y:(Double, String)) = if(x._1 < y._1) x else y
def checkPosition(latitude: Double, longitude: Double, borough: String): String = borough match{
  case n if borough == "bronx" => bronx.map( x => (getDistanceFromLatLonInKm(latitude,longitude, x.latitude, x.longitude),x.name))
                                                    .reduceLeft((x,y) => menor(x,y))._2
  case n if borough == "manhattan" => manhattan.map( x => (getDistanceFromLatLonInKm(latitude, longitude, x.latitude, x.longitude),x.name))
                                                              .reduceLeft((x,y) => menor(x,y))._2
  case n if borough == "queens" => queens.map( x => (getDistanceFromLatLonInKm(latitude, longitude, x.latitude, x.longitude),x.name) )
                                                        .reduceLeft((x,y) => menor(x,y))._2
  case n if borough == "brooklyn" => brooklyn.map( x => (getDistanceFromLatLonInKm(latitude, longitude, x.latitude, x.longitude),x.name) )
                                                            .reduceLeft((x,y) => menor(x,y))._2
  case n if borough == "staten island" => statenIsland.map( x => (getDistanceFromLatLonInKm(latitude, longitude, x.latitude, x.longitude),x.name)).reduceLeft((x,y) => menor(x,y))._2
}

case class info2( speed: Double, travel_time: Double, date: String, latitud: String, longitud: String, 
                  encoded_poly_line: String, borough: String, link_name: String, neighborhood: String,
                  humedad: String, presion: String, temperatura: String, descripcionClima: String, 
                  windDirection: String, windSpeed: String, accidentes: Int, heridos: Int, muertos: Int)





import java.util.Calendar
val dateFormatExpand = new java.text.SimpleDateFormat("yyyy-MM-dd hh:mm:ss")

case class data( manhattan: String, queens: String, brooklyn: String, statenIsland: String, bronx: String)
val cal = Calendar.getInstance()
def dates(x: org.apache.spark.sql.Row): List[(String,data)]={
  cal.setTime(dateFormatExpand.parse(x.getAs[String](0)))
  cal.add(Calendar.DAY_OF_MONTH, 29)
  cal.add(Calendar.YEAR, 5) 
  cal.add(Calendar.MONTH, 1)
  List( x.getAs[String](0) -> data( x.getAs[String](1), x.getAs[String](2), x.getAs[String](3), x.getAs[String](4), x.getAs[String](5)),
  outputformat.format(cal.getTime) -> data(x.getAs[String](1), x.getAs[String](2), x.getAs[String](3), x.getAs[String](4), x.getAs[String](5)))
}

val humedad = sqlContext.read.format("csv").option("header", "true").load("./humedad/humedad.csv")
val eHumedad = humedad.rdd.map(dates).flatMap(x => x).collect.toMap.withDefaultValue(data(null,null,null,null,null))

val pressure = sqlContext.read.format("csv").option("header", "true").load("./pressure/pressure.csv")
val ePressure = pressure.rdd.map(dates).flatMap(x => x).collect.toMap.withDefaultValue(data(null,null,null,null,null))

val temperature = sqlContext.read.format("csv").option("header", "true").load("./temperature/temperature.csv")
val eTemperature = temperature.rdd.map(dates).flatMap(x => x).collect.toMap.withDefaultValue(data(null,null,null,null,null))

val weatherDescription = sqlContext.read.format("csv").option("header", "true").load("./weatherDescription/weatherDescription.csv")
val eWeatherDescription = weatherDescription.rdd.map(dates).flatMap(x => x).collect.toMap.withDefaultValue(data(null,null,null,null,null))

val windDirection = sqlContext.read.format("csv").option("header", "true").load("./windDirection/windDirection.csv")
val eWindDirection = windDirection.rdd.map(dates).flatMap(x => x).collect.toMap.withDefaultValue(data(null,null,null,null,null))

val windSpeed = sqlContext.read.format("csv").option("header", "true").load("./windSpeed/windSpeed.csv")
val eWindSpeed = windSpeed.rdd.map(dates).flatMap(x => x).collect.toMap.withDefaultValue(data(null,null,null,null,null))


case class weather(humedad: String, presion: String, temperatura: String, descripcionClima: String, 
                   windDirection: String, windSpeed: String)

def addWeather(borough: String, fecha: String): weather=
  borough match{
    case "manhattan" => weather(eHumedad(fecha).manhattan, ePressure(fecha).manhattan,
                                    eTemperature(fecha).manhattan, eWeatherDescription(fecha).manhattan,
                                    eWindDirection(fecha).manhattan, eWindSpeed(fecha).manhattan)
    
    case "brooklyn" => weather(eHumedad(fecha).brooklyn, ePressure(fecha).brooklyn,
                                    eTemperature(fecha).brooklyn, eWeatherDescription(fecha).brooklyn,
                                    eWindDirection(fecha).brooklyn, eWindSpeed(fecha).brooklyn)
    
    case "queens" => weather(eHumedad(fecha).queens, ePressure(fecha).queens,
                                    eTemperature(fecha).queens, eWeatherDescription(fecha).queens,
                                    eWindDirection(fecha).queens, eWindSpeed(fecha).queens)
    
    case "bronx" => weather(eHumedad(fecha).bronx, ePressure(fecha).bronx,
                                    eTemperature(fecha).bronx, eWeatherDescription(fecha).bronx,
                                    eWindDirection(fecha).bronx, eWindSpeed(fecha).bronx)
    
    case "staten island" => weather(eHumedad(fecha).statenIsland, ePressure(fecha).statenIsland,
                                    eTemperature(fecha).statenIsland, eWeatherDescription(fecha).statenIsland,
                                    eWindDirection(fecha).statenIsland, eWindSpeed(fecha).statenIsland)
  }

  

val dateFormatAccidentes = new java.text.SimpleDateFormat("MM/dd/yyyy HH:mm")


val accidentes = sqlContext.read.format("csv").option("header", "true").load("./accidentes.csv")
case class accidentData (fecha: String, latitud: String, longitud: String, heridos: Int, muertos: Int, 
                         borough: String, street: String)

def parseable(fecha: String): String ={
  var str: String = null
  try{
      str = outputformat.format(dateFormatAccidentes.parse(fecha))
  }catch{
    case x: Exception => {return null}
    
  }
  return str; 
} 

val cleaned = accidentes.rdd.filter(x => parseable(x.getAs[String](0)+" "+x.getAs[String](1)) != null && 
                      x.getAs[String](2) != null &&
                      x.getAs[String](4) != null && 
                      x.getAs[String](5) != null &&
                      x.getAs[String](7) != null)

def sp(num: String): Int ={
  try{
    val number = num.toInt
    return number
  }catch{
    case x: Exception => {return 0}
  }
} 

def convert(x: org.apache.spark.sql.Row): accidentData={
  val fecha: String = parseable(x.getAs[String](0)+" "+x.getAs[String](1))
  val heridos: Int = sp(x.getAs[String](10))+sp(x.getAs[String](12))+sp(x.getAs[String](14))+sp(x.getAs[String](16))
  val muertos: Int = sp(x.getAs[String](11))+sp(x.getAs[String](13))+sp(x.getAs[String](15))+sp(x.getAs[String](17))
  accidentData(fecha, x.getAs[String](4),x.getAs[String](5), heridos, muertos, 
               x.getAs[String](2).trim.toLowerCase, x.getAs[String](7).trim.toLowerCase)
}

val accident_data = cleaned.map(convert)
case class accidentData2 (fecha: String, latitud: String, longitud: String, heridos: Int, muertos: Int, 
                          nAccidentes: Int, id: Long)
var i: Long = 0;
def grouped(x: ((String, String),Iterable[accidentData])): accidentData2 ={
    val nAccidentes = x._2.size
    val heridos = x._2.foldLeft(0)( (x,y) => x + y.heridos )
    val muertos = x._2.foldLeft(0)( (x,y) => x + y.muertos )
    val item = x._2.last
    i = i + 1
    accidentData2(x._1._1 ,item.latitud, item.longitud, heridos, muertos, nAccidentes, i)
}
val inmutableSet = accident_data.groupBy(x => (x.fecha,x.street)).map(grouped).collect.toSet


def procesAgruped(x:((Int, String), Iterable[info]) ): info2 ={
    val item = x._2.last
    val size = x._2.size
    val v_prom = x._2.foldLeft(0D)( (x,y) => x+y.speed.toDouble) / size
    val t_prom = x._2.foldLeft(0D)( (x,y) => x+y.travel_time.toDouble) / size
    val points = item.link_points.split(" ").map(_.split(","))
    val lat =  points(0)(0).toDouble
    val lon = points(0)(1).toDouble
    val check = mutableAccidents.filter(y => y.fecha == x._1._2)
    var accidentes = 0
    var muertos = 0
    var heridos = 0
    if(!check.isEmpty){
        val accident = check.map(y => (getDistanceFromLatLonInKm(y.latitud.toDouble,y.longitud.toDouble,
                                                             lat.toDouble,lon.toDouble), y))
                                  .reduceLeft((x,y) => if(x._1 < y._1) x else y)._2
        mutableAccidents -= accident
        accidentes = accident.nAccidentes
        muertos = accident.muertos
        heridos = accident.heridos
    }
    
    val w = addWeather(item.borough, x._1._2)
    info2(v_prom, t_prom, x._1._2, lat.toString, lon.toString, item.encoded_poly_line, 
          item.borough, item.link_name, checkPosition(lat, lon, item.borough), w.humedad, w.presion, 
          w.temperatura, w.descripcionClima, w.windDirection, w.windSpeed, accidentes, heridos, muertos)
}

val dataAgruped = contentRdd.groupBy(x => (x.link_id.toInt, x.date))

var a,b,c,d,e,f = ""
def checkAndReplace2(x: info2): info2 ={
  if(x.presion == null){
  return info2(
    x.speed,
    x.travel_time,
    x.date,
    x.latitud,
    x.longitud,
    x.encoded_poly_line,
    x.borough,
    x.link_name,
    x.neighborhood,
    a,
    b,
    c,
    d,
    e,
    f,
    x.accidentes,
    x.heridos,
    x.muertos)
  }else{
    a = x.humedad
    b = x.presion
    c = x.temperatura
    d = x.descripcionClima
    e = x.windDirection
    f = x.windSpeed
    return x
  }
}


val procesed = dataAgruped.map(procesAgruped(_)).map(checkAndReplace2).filter(x => !x.humedad.isEmpty || x.humedad != null)

procesed.toDF.repartition(1).write.option("header", "true").csv("./datasetFinalIntegrador2") 





In [ ]:
val procesedDataset = sqlContext.read.format("csv").option("header", "true").load("./procesedAccidents/procesedAccidents.csv")

case class info2( speed: String, travel_time: String, date: String, latitud: String, longitud: String, 
                  encoded_poly_line: String, borough: String, link_name: String, neighborhood: String,
                  humedad: String, presion: String, temperatura: String, descripcionClima: String, 
                  windDirection: String, windSpeed: String, accidentes: Int, heridos: Int, muertos: Int)

import scala.util.Random
def realAgrupedAccidents(x: ((String,String), Iterable[org.apache.spark.sql.Row])):List[info2] = {
    val shuffled = Random.shuffle(x._2)
    val item = shuffled.last
    val struct = shuffled.init
    val lat = item.getAs[String](3)
    val lon = item.getAs[String](4)
    val check = inmutableSet.filter(y => y.fecha == item.getAs[String](2))
    var accidentes = 0
    var muertos = 0
    var heridos = 0
    if(!check.isEmpty){
        val accident = check.map(y => (getDistanceFromLatLonInKm(y.latitud.toDouble,y.longitud.toDouble,
                                                             lat.toDouble,lon.toDouble), y))
                                  .reduceLeft((x,y) => if(x._1 < y._1) x else y)._2
       
        accidentes = accident.nAccidentes
        muertos = accident.muertos
        heridos = accident.heridos
    }
    val acc = info2(  
            item.getAs[String](0),
            item.getAs[String](1),
            item.getAs[String](2),
            item.getAs[String](3),
            item.getAs[String](4),
            item.getAs[String](5),
            item.getAs[String](6),
            item.getAs[String](7),
            item.getAs[String](8),
            item.getAs[String](9),
            item.getAs[String](10),
            item.getAs[String](11),
            item.getAs[String](12),
            item.getAs[String](13),
            item.getAs[String](14),
            accidentes,
            heridos,
            muertos
            )
    val finalStruct = acc :: (struct.map( y => info2(
        y.getAs[String](0),
        y.getAs[String](1),
        y.getAs[String](2),
        y.getAs[String](3),
        y.getAs[String](4),
        y.getAs[String](5),
        y.getAs[String](6),
        y.getAs[String](7),
        y.getAs[String](8),
        y.getAs[String](9),
        y.getAs[String](10),
        y.getAs[String](11),
        y.getAs[String](12),
        y.getAs[String](13),
        y.getAs[String](14),
        0,
        0,
        0)).toList)
    
    finalStruct.toList
}